In [36]:
import os
import math
import time
import json
import random
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from io import BytesIO

# Configuración 
url = "https://ovc.catastro.meh.es/INSPIRE/wfsBU.aspx"
output_format = "application/json"
output_filename = "edificios.geojson"
failed_tiles_file = "teselas_fallidas.json"
secciones_path = "C:\Users\ana_g\Desktop\TFM\capas secciones\catarroja_massanasa_sec.shp"

# Parámetros de teselado y reintentos
tile_size = 1000  # metros
max_retries = 5
base_timeout = 60  # segundos

# Leer límite de zona de estudio
limites = gpd.read_file(secciones_path)

# Crear teselas desde el bounding box
def create_tiles(minx, miny, maxx, maxy, size):
    tiles = []
    cols = math.ceil((maxx - minx) / size)
    rows = math.ceil((maxy - miny) / size)
    for i in range(cols):
        for j in range(rows):
            x1, y1 = minx + i * size, miny + j * size
            x2, y2 = min(x1 + size, maxx), min(y1 + size, maxy)
            tiles.append((x1, y1, x2, y2))
    return tiles

bbox = tuple(limites.total_bounds)
tiles = create_tiles(*bbox, tile_size)

# Descarga una tesela con reintentos, timeout variable y backoff con jitter (esto último porque el servidor está claro que se sobrecarga en determinadas horas
def fetch_tile(tile, retries=max_retries, base_delay=5):
    for attempt in range(1, retries + 1):
        timeout = base_timeout + (attempt - 1) * 30  # 60, 90, 120...
        try:
            print(f"  → Intento {attempt}/{retries} | Timeout: {timeout}s")
            params = {
                "SERVICE": "WFS",
                "VERSION": "2.0.0",
                "REQUEST": "GetFeature",
                "TYPENAMES": "BU.Building",
                "BBOX": f"{tile[0]},{tile[1]},{tile[2]},{tile[3]},EPSG:25830",
                "SRSNAME": "EPSG:25830",
                "OUTPUTFORMAT": output_format
            }
            r = requests.get(url, params=params, timeout=timeout)
            r.raise_for_status()
            gdf = gpd.read_file(BytesIO(r.content))
            return gdf if not gdf.empty else None
        except Exception as e:
            print(f"   Falló tesela {tile}: {e}")
            sleep_time = base_delay * attempt + random.uniform(1, 3)
            time.sleep(sleep_time)
    return None

# Descargar teselas
features = []
failed_tiles = []

print(f"\n Descargando {len(tiles)} teselas...\n")
for i, tile in enumerate(tiles):
    print(f"Tesela {i+1}/{len(tiles)}: {tile}")
    gdf = fetch_tile(tile)
    if gdf is not None:
        features.append(gdf)
    else:
        failed_tiles.append(tile)

# Reintentar las  teselas fallidas
if failed_tiles:
    print(f"\n Reintentando {len(failed_tiles)} teselas fallidas...\n")
    recovered = []
    still_failed = []

    for i, tile in enumerate(failed_tiles):
        print(f"Reintento {i+1}/{len(failed_tiles)}: {tile}")
        gdf = fetch_tile(tile)
        if gdf is not None:
            recovered.append(gdf)
        else:
            still_failed.append(tile)

    features.extend(recovered)
else:
    still_failed = []

# Guardar las que siguen dando fallo al descargar
if still_failed:
    with open(failed_tiles_file, "w") as f:
        json.dump(still_failed, f)
    print(f"\n  {len(still_failed)} teselas sin descargar.")
    print(f"   Guardadas en: {failed_tiles_file}")
else:
    if os.path.exists(failed_tiles_file):
        os.remove(failed_tiles_file)
    print("\n Todas las teselas descargadas correctamente.")

# Unir y procesar los resultados
if features:
    merged = gpd.GeoDataFrame(pd.concat(features, ignore_index=True))

    # Eliminar duplicados
    if 'gml_id' in merged.columns:
        merged = merged.drop_duplicates(subset='gml_id')
    else:
        merged = merged.drop_duplicates(subset='geometry')

    # Recortar con el límite de origen
    merged = gpd.overlay(merged, limites, how="intersection")

    # Guardar
    merged.to_file(output_filename, driver="GeoJSON")
    print(f"\n Edificios guardados en: {output_filename}")
    print(f" Total de edificios: {len(merged)}")
else:
    print("\n No se pudo descargar ningún dato.")


 Descargando 25 teselas...

Tesela 1/25: (718825.3853000002, 4365734.4113, 719825.3853000002, 4366734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 2/25: (718825.3853000002, 4366734.4113, 719825.3853000002, 4367734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 3/25: (718825.3853000002, 4367734.4113, 719825.3853000002, 4368734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 4/25: (718825.3853000002, 4368734.4113, 719825.3853000002, 4369734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 5/25: (718825.3853000002, 4369734.4113, 719825.3853000002, 4370105.134199999)
  → Intento 1/5 | Timeout: 60s
Tesela 6/25: (719825.3853000002, 4365734.4113, 720825.3853000002, 4366734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 7/25: (719825.3853000002, 4366734.4113, 720825.3853000002, 4367734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 8/25: (719825.3853000002, 4367734.4113, 720825.3853000002, 4368734.4113)
  → Intento 1/5 | Timeout: 60s
Tesela 9/25: (719825.3853000002, 4368734.4113, 720825.3853000002, 4369734.4113

In [38]:
gdf = gpd.read_file("edificios.geojson")
print(gdf.columns)
print(gdf.head())

Index(['gml_id', 'lowerCorner', 'upperCorner', 'beginLifespanVersion',
       'conditionOfConstruction', 'beginning', 'end', 'endLifespanVersion',
       'informationSystem', 'reference', 'localId', 'namespace',
       'horizontalGeometryEstimatedAccuracy',
       'horizontalGeometryEstimatedAccuracy_uom',
       'horizontalGeometryReference', 'referenceGeometry', 'currentUse',
       'numberOfBuildingUnits', 'numberOfDwellings',
       'numberOfFloorsAboveGround', 'documentLink', 'format', 'sourceStatus',
       'officialAreaReference', 'value', 'value_uom',
       'dateOfConstruction|DateOfEvent|beginning',
       'dateOfConstruction|DateOfEvent|end', 'beginning_', 'end_', 'CUSEC',
       'CUMUN', 'NMUN', 'geometry'],
      dtype='object')
                      gml_id           lowerCorner           upperCorner  \
0  ES.SDGC.BU.46246A01300066  719710.15 4366757.16  719794.27 4366826.32   
1  ES.SDGC.BU.000420100YJ16H  718856.14 4369659.44  718879.29 4369675.23   
2  ES.SDGC.BU.000540

In [40]:
gdf['currentUse'].value_counts()

currentUse
1_residential         2978
3_industrial           610
4_2_retail              70
2_agriculture           62
4_3_publicServices      62
4_1_office              26
Name: count, dtype: int64

In [42]:
# Cargar el archivo GML ya convertido en GeoDataFrame
gdf = gpd.read_file("edificios.geojson")  # O .gml si ya lo puedes leer directamente

# Filtrar por uso residencial
residenciales = gdf[gdf["currentUse"] == "1_residential"]

# Número de edificios residenciales
print(f"Número de edificios residenciales: {len(residenciales)}")

# Número total de viviendas (columna: numberOfDwellings)
total_viviendas = residenciales["numberOfDwellings"].fillna(0).astype(int).sum()
print(f"Número total de viviendas residenciales: {total_viviendas}")

Número de edificios residenciales: 2978
Número total de viviendas residenciales: 16700


In [44]:
print(gdf[["reference", "localId"]].head(10))

        reference         localId
0  46246A01300066  46246A01300066
1  000420100YJ16H  000420100YJ16H
2  000540100YJ16H  000540100YJ16H
3  000540200YJ16H  000540200YJ16H
4  001010100YJ16H  001010100YJ16H
5  001030100YJ16H  001030100YJ16H
6  001040200YJ16H  001040200YJ16H
7  46195A00100041  46195A00100041
8  46195A00100103  46195A00100103
9  46195A00100104  46195A00100104


In [46]:
# Cargar el archivo GeoJSON o GML
file_path = "edificios.geojson" 

# Cargar el archivo en un GeoDataFrame
gdf = gpd.read_file(file_path)

# Verificar las primeras filas del archivo para comprender la estructura
print(gdf.head())

# Agrupar por la columna que representa la parcela, 'reference' o 'localId'
# Utilizo 'reference', pero se podría cambiar, parece, a 'localId', si es necesario.
viviendas_por_parcela = gdf[gdf['currentUse'] == '1_residential'].groupby('reference').size()

# Mostrar número de viviendas por parcela
print(viviendas_por_parcela)

                      gml_id           lowerCorner           upperCorner  \
0  ES.SDGC.BU.46246A01300066  719710.15 4366757.16  719794.27 4366826.32   
1  ES.SDGC.BU.000420100YJ16H  718856.14 4369659.44  718879.29 4369675.23   
2  ES.SDGC.BU.000540100YJ16H  719774.69 4369411.55  719787.44 4369427.89   
3  ES.SDGC.BU.000540200YJ16H  719425.85 4369328.15  719465.74 4369361.87   
4  ES.SDGC.BU.001010100YJ16H  718963.26 4369164.69   718972.2 4369176.69   

  beginLifespanVersion conditionOfConstruction            beginning  \
0           2017-02-23              functional  1980/01/01 00:00:00   
1           2008-06-19              functional  1924/01/01 00:00:00   
2           2007-04-10              functional  1940/01/01 00:00:00   
3           2007-04-10              functional  1990/01/01 00:00:00   
4           2010-12-07              functional  1950/01/01 00:00:00   

                   end endLifespanVersion  \
0  1980/01/01 00:00:00               None   
1  1924/01/01 00:00:00    

In [48]:
# Cargar el archivo GeoJSON (o GML)
file_path = "edificios.geojson"  

# Cargar el archivo en un GeoDataFrame
gdf = gpd.read_file(file_path)

# Verificar primeras filas del archivo para asegurarse de que 'numberOfDwellings' existe
print(gdf.head())

# Agrupar por la columna 'reference' y sumar el número de viviendas
viviendas_por_parcela = gdf.groupby('reference')['numberOfDwellings'].sum()

# Mostrar el número de viviendas por parcela
print(viviendas_por_parcela)

                      gml_id           lowerCorner           upperCorner  \
0  ES.SDGC.BU.46246A01300066  719710.15 4366757.16  719794.27 4366826.32   
1  ES.SDGC.BU.000420100YJ16H  718856.14 4369659.44  718879.29 4369675.23   
2  ES.SDGC.BU.000540100YJ16H  719774.69 4369411.55  719787.44 4369427.89   
3  ES.SDGC.BU.000540200YJ16H  719425.85 4369328.15  719465.74 4369361.87   
4  ES.SDGC.BU.001010100YJ16H  718963.26 4369164.69   718972.2 4369176.69   

  beginLifespanVersion conditionOfConstruction            beginning  \
0           2017-02-23              functional  1980/01/01 00:00:00   
1           2008-06-19              functional  1924/01/01 00:00:00   
2           2007-04-10              functional  1940/01/01 00:00:00   
3           2007-04-10              functional  1990/01/01 00:00:00   
4           2010-12-07              functional  1950/01/01 00:00:00   

                   end endLifespanVersion  \
0  1980/01/01 00:00:00               None   
1  1924/01/01 00:00:00    

In [50]:
# Cargar datos SIN modificar valores nulos
gdf = gpd.read_file("edificios.geojson")

# Calcular viviendas por parcela (manteniendo NaN si no hay datos)
viviendas_por_parcela = (
    gdf.groupby('reference', dropna=False)['numberOfDwellings']
    .sum(min_count=1)  # Conserva NaN si todos son NaN
    .reset_index()
    .rename(columns={'numberOfDwellings': 'num_viv'})
)

# Disolver geometrías por 'reference'
parcelas_dissolved = gdf.dissolve(by='reference', as_index=False)

# Unir la suma de viviendas (left join para no perder registros)
parcelas_unidas = parcelas_dissolved.merge(
    viviendas_por_parcela,
    on='reference',
    how='left'
)

# Exportar a GeoPackage
output_file = "viviendas_parcelas.gpkg"
parcelas_unidas.to_file(output_file, driver='GPKG')

# Resumen
print(f"""
EXPORTACIÓN COMPLETADA: {output_file}
---------------------------------
* Parcelas únicas: {len(parcelas_unidas)}
* Viviendas totales: {parcelas_unidas['num_viv'].sum(skipna=True)}
* Parcelas con valores nulos: {parcelas_unidas['num_viv'].isna().sum()}
""")

# Muestra
print("\nMuestra de datos:")
print(parcelas_unidas[['reference', 'num_viv']].head())


EXPORTACIÓN COMPLETADA: viviendas_parcelas.gpkg
---------------------------------
• Parcelas únicas: 3788
• Viviendas totales: 16721
• Parcelas con valores nulos: 0


Muestra de datos:
           reference  num_viv     currentUse
3262  2976903YJ2627N      105  1_residential
1151  1182702YJ2618S       93  1_residential
1014  0887411YJ2608N       90  1_residential
3749  3081503YJ2638S       87  1_residential
3221  2876802YJ2627N       83  1_residential
